In [35]:
import pandas as pd
import numpy as np
from sklearn import preprocessing, feature_extraction

In [23]:
df = pd.DataFrame({'car':np.random.choice(['BMW', 'Mini', 'Mercedes'], size=10, replace=True)}, dtype='category')

In [24]:
df['car'].head()

0     BMW
1     BMW
2     BMW
3    Mini
4     BMW
Name: car, dtype: category
Categories (3, object): [BMW, Mercedes, Mini]

In [33]:
list(df.car.cat.codes)

[0, 0, 0, 2, 0, 2, 2, 1, 0, 1]

In [144]:
preprocessing.OneHotEncoder(sparse=False).fit_transform([[1],[2],[3],[1],[2],[3]])
#preprocessing.OneHotEncoder(sparse=False).fit_transform([1,2,3,1,2,3])

array([[ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.],
       [ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.]])

In [145]:
ohe = preprocessing.OneHotEncoder(sparse=False)

In [150]:
# ohe.fit_transform([[cod] for cod in df.car.cat.codes])
ohe.fit_transform(df.car.cat.codes.reshape(-1,1))

array([[ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 0.,  0.,  1.],
       [ 1.,  0.,  0.],
       [ 0.,  0.,  1.],
       [ 0.,  0.,  1.],
       [ 0.,  1.,  0.],
       [ 1.,  0.,  0.],
       [ 0.,  1.,  0.]])

In [151]:
df.car

0         BMW
1         BMW
2         BMW
3        Mini
4         BMW
5        Mini
6        Mini
7    Mercedes
8         BMW
9    Mercedes
Name: car, dtype: category
Categories (3, object): [BMW, Mercedes, Mini]

In [152]:
pd.Categorical(df.car, categories=['BMW', 'Mini'])

[BMW, BMW, BMW, Mini, BMW, Mini, Mini, NaN, BMW, NaN]
Categories (2, object): [BMW, Mini]

In [153]:
class OneHotEncoderPandasCategoricalSeries():
    def __init__(self, sparse=False, **kwargs):
        self.ohe = preprocessing.OneHotEncoder(handle_unknown='ignore', sparse=sparse, **kwargs)

    def _X_for_onehotenc(self, X):
        return X.cat.codes.reshape(-1,1 )
        # return [[x] for x in X.cat.codes]
    
    def fit(self, X, y=None):        
        self.categories = X.cat.categories
        self.ohe.fit(self._X_for_onehotenc(X))
        return self
        
        
    def transform(self, X, y=None):
        if np.all(X.cat.categories != self.categories):
            X = pd.Categorical(X, categories=self.categories)
        return self.ohe.transform(self._X_for_onehotenc(X))
        

In [154]:
ohec = OneHotEncoderPandasCategoricalSeries()

In [155]:
ohec.fit(df.car)

<__main__.OneHotEncoderPandasCategoricalSeries instance at 0x112db3a70>

In [156]:
ohec.transform(df.car)

array([[ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 0.,  0.,  1.],
       [ 1.,  0.,  0.],
       [ 0.,  0.,  1.],
       [ 0.,  0.,  1.],
       [ 0.,  1.,  0.],
       [ 1.,  0.,  0.],
       [ 0.,  1.,  0.]])

In [160]:
df2 = pd.DataFrame({'car':np.random.choice(['BMW', 'Mini', 'Mercedes','Ferrari'], size=10, replace=True)}, dtype='category')

In [161]:
df2

,car
0,Mercedes
1,Mercedes
2,BMW
3,Mini
4,Ferrari
5,BMW
6,Ferrari
7,Mini
8,Mini
9,Ferrari


In [159]:
ohec.transform(df2.car)

array([[ 0.,  0.,  1.],
       [ 0.,  0.,  1.],
       [ 0.,  1.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.],
       [ 1.,  0.,  0.],
       [ 0.,  0.,  1.],
       [ 0.,  0.,  1.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.]])